In [1]:
#data.table
library(data.table)
library(tictoc)
counts_dt <- fread("bulk_counts_long.csv")
meta_dt <- fread("sample_metadata.csv")

#inizio timer generale
tic("timer esercizio 4")

#TASK1
tic("task1")
annotated_dt <- counts_dt[meta_dt, on = "sample_id", nomatch = 0] #join, dove nomatch=0 garantisce che vengano incluse solo righe con una corrispondenza in meta_dt
#annotated_dt[, sample_id := NULL] #serve per rimuovere la colonna sample_id

per_patient_counts <- annotated_dt[, .(total_count = sum(count)), by = .(patient_id)] #ragggruppa per patient_id e gene
toc() #è la somma dei conteggi di quel gene su tutti i campioni prelevati da quel paziente.
print("Risultato dell'aggregazione per paziente:")
print(head(per_patient_counts))

#TASK2
tic("task2")

avg_counts_by_condition <- annotated_dt[, #media per count per condizione
    .(avg_count = mean(count)), #media per count=avg_count
    by = .(condition, gene) #media geni all'interno della condizione
]

top_10_genes <- avg_counts_by_condition[
    order(-avg_count), # Ordina 
    head(.SD, 10),     # Seleziona le prime 10 righe del sottodata.table (.SD)
    by = condition     #serve per visualizzare per condizione (separati)
]
toc()

print("Top 10 geni x condizione:")
print(top_10_genes)
toc()


Attaching package: ‘tictoc’


The following object is masked from ‘package:data.table’:

    shift




task1: 0.018 sec elapsed
[1] "Risultato dell'aggregazione per paziente:"
   patient_id total_count
       <char>       <int>
1:       P014       11159
2:       P007       10859
3:       P017       21645
4:       P044       11451
5:       P020       11057
6:       P036       11142
task2: 0.01 sec elapsed
[1] "Top 10 geni x condizione:"
    condition      gene avg_count
       <char>    <char>     <num>
 1:   control GENE_0272  179.3333
 2:   control GENE_0091  169.8889
 3:   control GENE_0425  160.7778
 4:   control GENE_0377  160.3333
 5:   control GENE_0262  147.3333
 6:   control GENE_0077  134.5556
 7:   control GENE_0259  130.2222
 8:   control GENE_0099  128.4444
 9:   control GENE_0495  127.1111
10:   control GENE_0417  114.5556
11:   treated GENE_0425  160.0000
12:   treated GENE_0272  158.6667
13:   treated GENE_0091  153.9333
14:   treated GENE_0377  148.0667
15:   treated GENE_0495  139.0000
16:   treated GENE_0262  130.8000
17:   treated GENE_0077  120.3333
18:   treated GEN

In [2]:
library(tictoc)
counts_df <- read.csv("bulk_counts_long.csv", stringsAsFactors = FALSE)
meta_df <- read.csv("sample_metadata.csv", stringsAsFactors = FALSE)

# Inizio timer generale
tic("timer esercizio 4")

#TASK1
tic("task1")

# Join tra i due data frame in base a sample_id
merged_df <- merge(counts_df, meta_df, by = "sample_id")

# Aggregazione per paziente e gene (somma dei count)
per_patient_counts_df <- aggregate(
  merged_df$count,
  by = list(patient_id = merged_df$patient_id, gene = merged_df$gene),
  FUN = sum
)
colnames(per_patient_counts_df)[3] <- "total_count"

toc()
cat("Risultato dell'aggregazione per paziente:\n")
print(head(per_patient_counts_df))

#TASK2
tic("task2")

# Calcolo della media per condition e gene
avg_df <- aggregate(
  merged_df$count,
  by = list(condition = merged_df$condition, gene = merged_df$gene),
  FUN = mean
)
colnames(avg_df)[3] <- "avg_count"

# Ordinamento e selezione dei top 10 geni per condizione
top_10_genes_list <- list()
conditions <- unique(avg_df$condition)

for (cond in conditions) {
  subset_df <- avg_df[avg_df$condition == cond, ]
  subset_df <- subset_df[order(-subset_df$avg_count), ]  # ordina decrescente
  top_10_genes_list[[cond]] <- head(subset_df, 10)
}

# Combina tutto in un unico data frame
top_10_genes_df <- do.call(rbind, top_10_genes_list)
rownames(top_10_genes_df) <- NULL

toc()
cat("Top 10 geni per condizione:\n")
print(top_10_genes_df)

# Fine timer generale
toc()

task1: 0.056 sec elapsed
Risultato dell'aggregazione per paziente:
  patient_id      gene total_count
1       P002 GENE_0000          17
2       P007 GENE_0000           9
3       P008 GENE_0000          15
4       P009 GENE_0000           6
5       P011 GENE_0000          13
6       P013 GENE_0000           5
task2: 0.019 sec elapsed
Top 10 geni per condizione:
   condition      gene avg_count
1    control GENE_0272  179.3333
2    control GENE_0091  169.8889
3    control GENE_0425  160.7778
4    control GENE_0377  160.3333
5    control GENE_0262  147.3333
6    control GENE_0077  134.5556
7    control GENE_0259  130.2222
8    control GENE_0099  128.4444
9    control GENE_0495  127.1111
10   control GENE_0417  114.5556
11   treated GENE_0425  160.0000
12   treated GENE_0272  158.6667
13   treated GENE_0091  153.9333
14   treated GENE_0377  148.0667
15   treated GENE_0495  139.0000
16   treated GENE_0262  130.8000
17   treated GENE_0077  120.3333
18   treated GENE_0099  116.7333
19   tre

In [4]:
#tabella confronto tempi
T_DT1 <- 0.018
T_DT2 <- 0.010
T_DF1 <- 0.056
T_DF2 <- 0.019

# Creazione della tabella riassuntiva
risultati_performance <- data.frame(
  Task = c("Task 1",
           "Task 2"),
  
  Tempo_data.table_Sec = c(T_DT1, T_DT2),
  Tempo_data.frame_Sec = c(T_DF1, T_DF2)
)

# Aggiungiamo una colonna per il fattore di velocizzazione (Speedup)
risultati_performance$Speedup_DT_vs_DF <- 
  round(risultati_performance$Tempo_data.frame_Sec / risultati_performance$Tempo_data.table_Sec, 1)

# Stampiamo la tabella finale
print(risultati_performance)

    Task Tempo_data.table_Sec Tempo_data.frame_Sec Speedup_DT_vs_DF
1 Task 1                0.018                0.056              3.1
2 Task 2                0.010                0.019              1.9
